In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 设置中文字体为宋体
plt.rcParams['font.sans-serif'] = ['SimHei']
# 设置英文字体为新罗马
plt.rcParams['font.serif'] = ['Times New Roman']
# 字体大小
plt.rcParams['font.size'] = 12  
# 正常显示负号
plt.rcParams['axes.unicode_minus'] = False 

In [3]:
def result(file):
    df = pd.read_csv(file, encoding='gbk').drop(["单品编码", "分类编码"], axis=1)

    # 计算平均利润
    df['平均利润'] = (df['销售单价(元/千克)'] - df['批发价格(元/千克)']) * df['销量(千克)']
    # 计算平均利润率
    df['平均利润率'] = (df['销售单价(元/千克)'] - df['批发价格(元/千克)']) / df['批发价格(元/千克)']
    # 计算加成率
    df['加成率'] = df['平均利润率'] / (1 - df['平均利润率'])
    # 计算成本加成定价
    df['成本加成定价'] = df['批发价格(元/千克)'] * (1 + df['加成率'])

    # 如果成本加成定价大于批发价的1.75倍，则将成本加成定价设置为批发价的1.75倍
    #max_allowed_price = df['批发价格(元/千克)'] * 1.75
    #df['成本加成定价'] = df['成本加成定价'].where(df['成本加成定价'] <= max_allowed_price, max_allowed_price)

    df['成本加成定价'].fillna(0, inplace=True)

    return df

def data_prepare(df):
    
    df['销售日期'] = pd.to_datetime(df['销售日期'])
    df = df.resample('D', on='销售日期').agg({'销量(千克)': 'sum', '成本加成定价': 'mean', '销售单价(元/千克)': 'mean'})   # ------------------------注意啊这里是以周为单位而不是月
    df['日期'] = df.index
    df['成本加成定价'].fillna(0, inplace=True)
    # 将DataFrame中的所有inf值替换为0
    df.replace([np.inf, -np.inf], 0, inplace=True)
    return df

df1 = result("data_2\\花菜类.csv")
df2 = result("data_2\\花叶类.csv")
df3 = result("data_2\\茄类.csv")
df4 = result("data_2\\辣椒类.csv")
df5 = result("data_2\\食用菌.csv")
df6 = result("data_2\\水生根茎类.csv")
name = ["花菜类", "花叶类", "茄类", "辣椒类", "食用菌", "水生根茎类"]



df1_res = data_prepare(df1)
df2_res = data_prepare(df2)
df3_res = data_prepare(df3)
df4_res = data_prepare(df4)
df5_res = data_prepare(df5)
df6_res = data_prepare(df6)
df1_res.head()

from sklearn.ensemble import IsolationForest

def deal_out(df1):
    model = IsolationForest()
    df1['is_outlier'] = model.fit_predict(df1[['成本加成定价']])
    df_cleaned = df1[df1['is_outlier'] != -1]
    return df_cleaned

df1_res = deal_out(df1_res)
df2_res = deal_out(df2_res)
df3_res = deal_out(df3_res)
df4_res = deal_out(df4_res)
df5_res = deal_out(df5_res)
df6_res = deal_out(df6_res)

In [4]:
df1_res.head()

销量(千克)     成本加成定价  销售单价(元/千克)         日期  is_outlier
销售日期                                                            
2020-07-01  46.640  17.419473   12.834951 2020-07-01           1
2020-07-02  43.943  18.978969   12.421053 2020-07-02           1
2020-07-03  42.076  18.374880   12.000000 2020-07-03           1
2020-07-04  55.662  18.167197   12.619048 2020-07-04           1
2020-07-05  55.474  17.002282   12.641509 2020-07-05           1

<br><br><br>

# 确定约束条件：时间序列的置信度


置信区间是一个统计概念，用于表示对于一个参数或统计量的估计值的不确定性范围。它并不直接表示预测值的范围，而是关于估计值的一个区间，使得我们可以合理地估计真实值可能位于该区间内的概率。

具体来说，如果我们计算了一个参数的置信区间，例如均值的置信区间，那么该区间表示了我们对真实均值所持的一定程度的信心。例如，95%的置信区间意味着在多次重复抽样的情况下，我们预计约有95%的情况下，真实均值会落在该区间内。这并不意味着在任何特定情况下预测值都会落在这个区间内，因为每个样本可能都会有一定的随机性。

置信区间通常与统计假设检验相关，它们用于帮助我们评估样本数据中的估计值（如均值、方差等）的可靠性，并提供了一种衡量估计的不确定性的方式。更广泛地说，置信区间是统计推断中的一个重要工具，用于从样本数据中获取关于总体参数的信息。

总之，置信区间表示了对于估计值的不确定性，并告诉我们在一定置信水平下真实值可能位于该区间内。但它并不直接表示预测值的范围，因为预测值通常还涉及到更多的模型和假设。

<br><br><br><br><br>

确定时间序列的置信区间通常涉及统计分析方法，其中最常见的方法之一是使用基于统计模型的方法，例如ARIMA（自回归集成滑动平均）、GARCH（广义自回归条件异方差）等。下面是一个简单的方法，使用移动平均和标准差来估计时间序列的置信区间：

计算移动平均：首先，计算时间序列的移动平均。移动平均可以帮助平滑时间序列，减少噪音的影响。你可以选择不同的窗口大小，例如3天、7天或30天的移动平均，具体取决于你的数据和需求。

计算标准差：接下来，计算时间序列的标准差。标准差衡量了数据点的离散程度，较大的标准差表示数据波动较大，较小的标准差表示数据波动较小。

计算置信区间：一旦你有了移动平均和标准差，你可以使用统计原理来计算置信区间。通常情况下，你可以使用正态分布或t分布来估计置信区间，具体取决于你的样本大小和数据的性质。常见的置信水平包括95%和99%。

对于正态分布，你可以使用以下公式来计算置信区间：
置信区间 = 移动平均 ± （Z值 * 标准差 / sqrt(样本大小)）
其中，Z值是正态分布对应于所选置信水平的分位数，例如95%置信水平对应的Z值约为1.96。

对于t分布，你可以使用以下公式来计算置信区间：
置信区间 = 移动平均 ± （t值 * 标准差 / sqrt(样本大小)）
其中，t值是t分布对应于所选置信水平和自由度的分位数。

请注意，这只是一个简单的方法，适用于某些情况下。如果你的时间序列具有复杂的季节性、趋势或异方差性质，或者你需要更高级的置信区间估计方法，那么可能需要使用更复杂的统计模型，或者考虑使用专业的时间序列分析工具和软件。此外，选择合适的置信水平也要考虑你的具体需求和风险偏好。

In [5]:
import numpy as np
from scipy.stats import norm, t


def find_bound(df, name):
    # 计算移动平均和标准差（这里以7天的窗口为例）
    window_size = 7
    moving_avg = np.convolve(df[name], np.ones(window_size)/window_size, mode='valid')
    std_dev = np.std(df[name])

    # 选择置信水平（例如，95%）
    confidence_level = 0.6

    # 计算置信区间的半宽度
    if len(df[name]) >= 30:
        # 使用正态分布
        z_value = norm.ppf((1 + confidence_level) / 2)
        margin_error = z_value * (std_dev / np.sqrt(len(df[name])))
    else:
        # 使用t分布，如果样本大小较小
        degrees_of_freedom = len(df1_res[name]) - 1
        t_value = t.ppf((1 + confidence_level) / 2, df=degrees_of_freedom)
        margin_error = t_value * (std_dev / np.sqrt(len(df[name])))

    # 计算置信区间
    lower_bound = min(moving_avg - margin_error)
    upper_bound = max(moving_avg + margin_error)

    return lower_bound, upper_bound

lower_bound_1_1, upper_bound_1_1 = find_bound(df1_res, '销量(千克)')
lower_bound_1_2, upper_bound_1_2 = find_bound(df1_res, '成本加成定价')

lower_bound_2_1, upper_bound_2_1 = find_bound(df2_res, '销量(千克)')
lower_bound_2_2, upper_bound_2_2 = find_bound(df2_res, '成本加成定价')

lower_bound_3_1, upper_bound_3_1 = find_bound(df3_res, '销量(千克)')
lower_bound_3_2, upper_bound_3_2 = find_bound(df3_res, '成本加成定价')

lower_bound_4_1, upper_bound_4_1 = find_bound(df4_res, '销量(千克)')
lower_bound_4_2, upper_bound_4_2 = find_bound(df4_res, '成本加成定价')

lower_bound_5_1, upper_bound_5_1 = find_bound(df5_res, '销量(千克)')
lower_bound_5_2, upper_bound_5_2 = find_bound(df5_res, '成本加成定价')

lower_bound_6_1, upper_bound_6_1 = find_bound(df6_res, '销量(千克)')
lower_bound_6_2, upper_bound_6_2 = find_bound(df6_res, '成本加成定价')

In [6]:
print(lower_bound_1_1, upper_bound_1_1)
print(lower_bound_1_2, upper_bound_1_2)

7.651135431721546 115.74929313970702
0.22561647834573037 23.4072820568684


In [7]:
print(lower_bound_2_1, upper_bound_2_1)
print(lower_bound_2_2, upper_bound_2_2)

40.5780896021723 467.2400532549705
-0.5154517083269119 16.9385818903737


In [8]:
print(lower_bound_3_1, upper_bound_3_1)
print(lower_bound_3_2, upper_bound_3_2)

-0.359321038246891 57.71317818110403
-1.6126438868670474 30.65420445524721


In [9]:
print(lower_bound_4_1, upper_bound_4_1)
print(lower_bound_4_2, upper_bound_4_2)

3.809870312049301 222.8035582593793
-1.9697415567128158 31.590038448727142


In [10]:
print(lower_bound_5_1, upper_bound_5_1)
print(lower_bound_5_2, upper_bound_5_2)

8.518161430301763 250.96069571255538
2.6515760144852165 41.27447161904096


In [11]:
print(lower_bound_6_1, upper_bound_6_1)
print(lower_bound_6_2, upper_bound_6_2)

1.132729241695602 130.54298504401868
-0.173388930794969 27.56531732310192
